In [1]:
# Importing the importations 
import pandas as pd
import numpy as np
#import dataframe as df
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
from keras.regularizers import l2, l1
from keras.models import load_model
import os
from keras.layers.advanced_activations import PReLU
from keras.optimizers import *
import keras
from keras.layers import *
from keras.models import Sequential

# All tqdm instances can be removed without breaking the code.
from tqdm import tqdm_notebook as tq

# To see the training error in real time.
from keras import callbacks
remote = callbacks.RemoteMonitor(root='http://localhost:9000')

D:\Software\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.
Using gpu device 0: GeForce GTX 960M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)


In [7]:
## Reading the input data
raw_data = pd.read_csv("train.csv",delimiter=",")
raw_data_test = pd.read_csv("test.csv",delimiter=",")

In [8]:
raw_data.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


We add a columb to the test set because it's easier to take care of in the preprocessing part.

In [9]:
raw_data_test["loss"] = pd.Series([0]*len(raw_data_test))
raw_data_test.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,4,A,B,A,A,A,A,A,A,B,...,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562,0
1,6,A,B,A,B,A,A,A,A,B,...,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045,0
2,9,A,B,A,B,B,A,B,A,B,...,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232,0
3,12,A,A,A,A,B,A,A,A,A,...,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955,0
4,15,B,A,A,A,A,B,A,A,A,...,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823,0


The preprocessing part. Just one hot encoding. It can be also noted that if a test sample happens to have a unknown (new) class, we replace that by the most common class.

In [10]:
class Preprocess:
    
    def __init__(self, dataset_to_fit):
        self.encoder = []
        self.onehot = OneHotEncoder()
        self.maxloss = dataset_to_fit["loss"].max()
        for i, feature in enumerate(tq(dataset_to_fit.columns)):
            if 'cat' in feature:
                self.encoder.append(list(dataset_to_fit[feature].value_counts().index))
                
        temp_data = self.conversion1(dataset_to_fit)
        
        self.onehot.fit(temp_data[:,1:117])
    

    def conversion1(self,dataframe):
        new_dataframe = pd.DataFrame()
        
        for i, feature in enumerate(tq(dataframe.columns)):
            if 'cat' in feature:
                new_dataframe[feature]=dataframe[feature].map(lambda x: self.convert(i,x))
            else:
                new_dataframe[feature] = dataframe[feature]
        
        return new_dataframe.as_matrix()
    
    def conversion2(self,array):
        indexes = array[:,0]
        cat = self.onehot.transform(array[:,1:117])
        cont = array[:,117:-1]
        loss = array[:,-1]
                
        return indexes, sparse.hstack((cont,cat)).toarray(), loss
        
    def convert(self,i,element):
        _list = self.encoder[i-1]
        try:
            return _list.index(element)
        except ValueError:
            return 0
    
    
    def convert_back(self,i,element):
        return self.encoder[i-1][element]
        
        
    def process(self,dataframe):
        array = self.conversion1(dataframe)
        indexes, features,loss = self.conversion2(array)
        
        #loss = loss/self.maxloss
        
        # Each one of them is a numpy array
        return indexes, features, loss

In [11]:
prepro = Preprocess(raw_data)

In [12]:
indexes, features,loss = prepro.process(raw_data)

The model used. Quite simple.

In [13]:
model = Sequential()
model.add(Dense(256,activation = "relu", W_regularizer = l2(.01), input_dim = 1153))
model.add(Dense(128,activation = "relu", W_regularizer = l2(.01)))
model.add(Dense(64,activation = "relu", W_regularizer = l2(.01)))
model.add(Dense(32,activation = "relu", W_regularizer = l2(.01)))
model.add(Dense(16,activation = "relu", W_regularizer = l2(.01)))
model.add(Dense(1, init = 'he_normal'))

In [14]:
#sgd = SGD(lr=0.02, momentum=0.0, decay=0.03)
adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.003)

model.compile(optimizer=adam, loss='mae')

In [15]:
try:
    model.fit(features, loss, batch_size=64, nb_epoch=1000, verbose=0, callbacks=[remote], validation_split=0.1)
except KeyboardInterrupt:
    pass

Saving and loading models.

In [19]:
model.save("relu_regularizer.m5")

OSError: Unable to create file (Unable to truncate a file which is already open)

In [21]:
model = load_model("relu_regularizer.m5")

Generating the submission:

In [ ]:
indexes_test, features_test,loss_test = prepro.process(raw_data_test)

In [ ]:
losses_predicted = model.predict(features_test)

In [ ]:
indexes = list(indexes_test)
indexes = [int(x) for x in indexes]

In [ ]:
loss_ = list(losses_predicted)
loss_ = [x[0] for x in loss_]

In [ ]:
submission_list = zip(indexes, loss_)

In [ ]:
submission_number = 0
while os.path.isfile("submission_" + str(submission_number) + ".csv"):
    submission_number += 1
    
    
f = open("submission_" + str(submission_number) + ".csv", 'w')
f.write("id,loss\n")
for tup in submission_list:
    f.write(str(tup[0]) + "," + str(tup[1]) + "\n")
f.close()
    
print("Submission file number " + str(submission_number) + " was created")